In [0]:
from sklearn.datasets import load_digits
digits = load_digits()

In [0]:
# Print to show there are 1797 images (8 by 8 images for a dimensionality of 64)
print("Image Data Shape" , digits.data.shape)

# Print to show there are 1797 labels (integers from 0-9)
print("Label Data Shape", digits.target.shape)

In [0]:
import numpy as np 
import matplotlib.pyplot as plt

nen=9

plt.figure(figsize=(20,nen-1))
for index, (image, label) in enumerate(zip(digits.data[0:nen], digits.target[0:nen])):
    plt.subplot(1, nen, index + 1)
    plt.imshow(np.reshape(image, (8,8)), cmap=plt.cm.gray)
    plt.title('Training: %i\n' % label, fontsize = 20)

In [0]:
import numpy as np 

def sigmoid(x):
    return 1/(1+np.exp(-x))

def computeCost_reg(theta, X, y, Lambda=1, return_grad = False):

    m = len(y) # number of training examples

    J=0
    grad = np.zeros(theta.shape)
    theta0 = np.copy(theta)
    theta0[0] = 0

    h_theta=sigmoid(np.dot(theta.T, X.T))

    left = -y*np.log(h_theta)
    right = -(1-y)*np.log(1-h_theta)
    rez = left + right
    J = 1/m * np.sum(rez) + Lambda/(2*m) * np.dot(theta0.T, theta0)

    grad = 1/m*(np.dot(X.T, (h_theta-y).T) + Lambda/m * theta0)
    if (return_grad):
        return [J, grad]
    else:
        return J

In [0]:
from scipy.optimize import minimize

def oneVsAll(X, y, num_labels, Lambda):
    m,n = X.shape
    all_theta = np.zeros((num_labels, n+1))
    X = np.c_[np.ones((m, 1)),X]
    initial_theta = np.zeros((n+1, 1))


    for c in range(num_labels):
        myargs = (X, (y%10==c).astype(int), Lambda, True)
        theta = minimize(computeCost_reg, x0=initial_theta, args=myargs, options={'disp': True, 'maxiter':13}, method="Newton-CG", jac=True)
        # theta = minimize(computeCost_reg, x0=initial_theta, args=myargs, options={'disp': True, 'maxiter':20}, method="CG", jac=True)
        all_theta[c, :] = theta["x"]
    return all_theta

In [0]:
Lambda = 1
input_layer_size = 400
num_labels = 10
all_theta = oneVsAll(digits.data, digits.target, num_labels, Lambda)

In [0]:
all_theta[:, 0]

In [0]:
def predictOneVsAll(all_theta, X):
    m,_ = X.shape
    num_labels,_ = all_theta.shape
    p = np.zeros((m, 1))
    X = np.c_[np.ones((m, 1)), X]
    p = np.argmax(sigmoid( np.dot(X, all_theta.T) ), axis=1)
    return p

In [0]:
pred = predictOneVsAll(all_theta, digits.data)
print('Training Set Accuracy: {:f}'.format((np.mean(pred == digits.target)*100)))